---
title: Minimum variance analysis (MVA)
---

Notes:

- To ensure the accuracy of MVA, only when the ratio of the middle to the minimum eigenvalue (labeled QMVA for simplicity) is larger than 3 are the results used for further analysis.


In [88]:
# | default_exp propeties/mva
# | export
import xarray as xr
import numpy as np
import pandas as pd
from datetime import timedelta

from lmfit.models import StepModel, ConstantModel, Model
from lmfit import Parameters

The following method implicitly assumes the data is evenly sampled, otherwise, data resampling is needed.


In [89]:
# | exports
def minvar(data):
    """
    see `pyspedas.cotrans.minvar`

    This program computes the principal variance directions and variances of a
    vector quantity as well as the associated eigenvalues.

    Parameters
    -----------
    data:
        Vxyz, an (npoints, ndim) array of data(ie Nx3)

    Returns
    -------
    vrot:
        an array of (npoints, ndim) containing the rotated data in the new coordinate system, ijk.
        Vi(maximum direction)=vrot[0,:]
        Vj(intermediate direction)=vrot[1,:]
        Vk(minimum variance direction)=Vrot[2,:]
    v:
        an (ndim,ndim) array containing the principal axes vectors
        Maximum variance direction eigenvector, Vi=v[*,0]
        Intermediate variance direction, Vj=v[*,1] (descending order)
    w:
        the eigenvalues of the computation
    """

    #  Min var starts here
    # data must be Nx3
    vecavg = np.nanmean(np.nan_to_num(data, nan=0.0), axis=0)

    mvamat = np.zeros((3, 3))
    for i in range(3):
        for j in range(3):
            mvamat[i, j] = (
                np.nanmean(np.nan_to_num(data[:, i] * data[:, j], nan=0.0))
                - vecavg[i] * vecavg[j]
            )

    # Calculate eigenvalues and eigenvectors
    w, v = np.linalg.eigh(mvamat, UPLO="U")

    # Sorting to ensure descending order
    w = np.abs(w)
    idx = np.flip(np.argsort(w))

    # IDL compatability
    if True:
        if np.sum(w) == 0.0:
            idx = [0, 2, 1]

    w = w[idx]
    v = v[:, idx]

    # Rotate intermediate var direction if system is not Right Handed
    YcrossZdotX = v[0, 0] * (v[1, 1] * v[2, 2] - v[2, 1] * v[1, 2])
    if YcrossZdotX < 0:
        v[:, 1] = -v[:, 1]
        # v[:, 2] = -v[:, 2] # Should not it is being flipped at Z-axis?

    # Ensure minvar direction is along +Z (for FAC system)
    if v[2, 2] < 0:
        v[:, 2] = -v[:, 2]
        v[:, 1] = -v[:, 1]

    vrot = np.array([np.dot(row, v) for row in data])

    return vrot, v, w

## MVA related features


In [90]:
# | export
def calc_mva_features(data: np.ndarray):
    """
    Compute MVA features based on the given data array.

    Parameters:
    - data (np.ndarray): Input data

    Returns:
    - List: Computed features
    """

    # Compute variance properties
    vrot, v, eigs = minvar(data)

    # Maximum variance direction eigenvector
    Vl = v[:, 0]
    Vn = v[:, 2]

    vec_mag = np.linalg.norm(vrot, axis=1)

    # Compute changes in each component of B_rot
    dvec = [vrot[0, i] - vrot[-1, i] for i in range(3)]

    # Compute mean values
    vec_mag_mean = np.mean(vec_mag)
    vec_n_mean = np.mean(vrot[:, 2])
    VnOverVmag = vec_n_mean / vec_mag_mean

    # Compute relative changes in magnitude
    dvec_mag = vec_mag[-1] - vec_mag[0]
    dBOverB = np.abs(dvec_mag / vec_mag_mean)
    dBOverB_max = (np.max(vec_mag) - np.min(vec_mag)) / vec_mag_mean

    output_names = [
        "Vl",
        "Vn",
        "b_mag",
        "b_n",
        "db_mag",
        "bn_over_b",
        "db_over_b",
        "db_over_b_max",
        "dB_lmn",
    ]

    results = [
        Vl,
        Vn,
        vec_mag_mean,
        vec_n_mean,
        dvec_mag,
        VnOverVmag,
        dBOverB,
        dBOverB_max,
        dvec,
    ]

    return pd.Series(results, index=output_names), vrot

## Fit maximum variance direction


$$
f(x; A, \mu, \sigma, {\mathrm{form={}'logistic{}'}}) = A \left[1 - \frac{1}{1 + e^{\alpha}} \right]
$$

where $\alpha = (x - \mu)/{\sigma}$. And the derivative is

$$
\frac{df}{dx} = \frac{A}{\sigma} \frac{e^{\alpha}}{(1 + e^{\alpha})^2}
$$

at center $x = \mu$, the derivative is

$$
\frac{df}{dx} = \frac{A}{4 \sigma}
$$


In [55]:
# | export
def fit_maxiumum_variance_direction(
    ts: xr.DataArray, datetime_unit="s", return_best_fit: bool = True, **kwargs
):
    """
    Fit maximum variance direction data by model

    Note: 
        - see `datetime_to_numeric` in `xarray.core.duck_array_ops` for more details about converting datetime to numeric
        - Xarray uses the numpy dtypes datetime64[ns] and timedelta64[ns] to represent datetime data.
    """
    time = ts["time"].values
    x = (time - min(time)) / np.timedelta64(1, datetime_unit)
    y = ts.values

    xmin, xmax = min(x), max(x)
    ymin, ymax = min(y), max(y)

    # Create a model
    step_mod = StepModel(form="logistic")
    const_mod = ConstantModel()
    mod = step_mod + const_mod

    # Create parameters
    params = Parameters()
    params.add("c", value=0)
    params.add("center", value=(xmax + xmin) / 2.0, min=xmin, max=xmax)
    params.add("amplitude", value=(ymax - ymin))
    params.add("sigma", value=(xmax - xmin) / 7.0, min=0)

    out = mod.fit(y, params, x=x)

    amplitude = out.params["amplitude"].value
    sigma = out.params["sigma"].value
    center = out.params["center"].value
    max_df = amplitude / (4 * sigma)

    d_time = min(time) + center * np.timedelta64(1, datetime_unit).astype("timedelta64[ns]")

    result = pd.Series(
        {
            "fit.vars.amplitude": amplitude,
            "fit.vars.sigma": sigma,
            "d_time": d_time,
            "d_star": max_df,
            "fit.vars.c": out.params["c"].value,
            "fit.stat.rsquared": out.rsquared,
            "fit.stat.chisqr": out.chisqr,
        }
    )
    if return_best_fit:
        result["fit.best_fit"] = out.best_fit
        result["fit.time"] = time
    return result

In [56]:
# | export
def calc_candidate_mva_features(event, data: xr.DataArray, **kwargs):
    event_data = data.sel(time=slice(event["d_tstart"], event["d_tstop"]))

    mva_features, vrot = calc_mva_features(event_data.to_numpy())

    event_data_l = xr.DataArray(
        vrot[:, 0], dims=["time"], coords={"time": event_data.time}
    )

    fit_result = fit_maxiumum_variance_direction(event_data_l, **kwargs)

    return pd.concat([mva_features, fit_result])

## Test


In [99]:
# | code-summary: test for ts_max_distance function
time = pd.date_range("2000-01-01", periods=10)
x = np.linspace(0, np.pi, 10)
# generate data circular in three dimensions, so the biggest distance is between the first and the last point
data = np.zeros((10, 3))
data[:, 0] = np.cos(x)
data[:, 1] = np.sin(x)
ts = xr.DataArray(data, coords={"time": time}, dims=["time", "space"])
fit_ts(ts[:, 0]).best_fit

array([ 1.01476962,  0.92001526,  0.75955714,  0.5124786 ,  0.18267077,
       -0.18267075, -0.51247858, -0.75955714, -0.92001527, -1.01476964])

In [120]:
np.timedelta64()

numpy.timedelta64(1000000,'us')

In [73]:
mva_features, vrot = calc_mva_features(data)
vrot.shape

(10, 3)

In [ ]:
# | eval: false
from fastcore.test import test_eq

# Generate synthetic data
np.random.seed(42)  # for reproducibility
data = np.random.rand(100, 3)  # 100 time points, 3-dimensional data
# Call the mva_features function
features, vrot = calc_mva_features(data)
_features = [
    0.3631060892452051,
    0.8978455426527485,
    -0.24905290500542857,
    0.09753158579102299,
    0.086943767300213,
    0.07393142040422575,
    1.1760056390752571,
    0.9609421690770317,
    0.6152039820297959,
    -0.5922397773398479,
    0.6402091632847049,
    0.61631157045453,
    1.2956351134759623,
    0.19091785005728523,
    0.5182488424049534,
    0.4957624347593598,
]
test_eq(features, _features)